# 2. Swin Transformer (2021) – Классификация/Сегментация
# Описание:
**Swin Transformer – это трансформер для изображений, который разбивает картинку на "окна" (windows) для эффективного анализа. Работает лучше CNN в задачах типа классификации, детекции и сегментации.**

# Kaggle/Источник:


*   Оригинальная статья (2021) - https://arxiv.org/abs/2103.14030
*   Пример реализации на PyTorch - https://www.kaggle.com/code/awsaf49/swin-transformer-tf-tutorial

# Ключевые особенности:



# 1. Иерархическая структура (как у CNN):


*   Изображение делится на патчи (например, 4×4 пикселя), которые преобразуются в токены (как в NLP).

*   На каждом уровне окна становятся больше, а разрешение уменьшается (аналогично пулингу в CNN).


# 2. Shifted Windows (Сдвигаемые окна):

*   На первом слое окна разбиты без перекрытия.

*   На следующем слое окна сдвигаются на половину размера, чтобы модель видела связи между разными областями.


# 3. Self-Attention внутри окон:

*   Вместо глобального внимания (как в ViT), Swin Transformer считает attention только внутри каждого окна, что сильно ускоряет работу.

# Как предсказывает класс изображения?


1.   Изображение → разбивается на токены → проходится через Swin-блоки (чередуются обычные и shifted windows).

2.   В конце усредняются фичи всех токенов → подаются в классификатор (аналогично EfficientNetV2).

# Почему лучше CNN?



* Трансформеры лучше улавливают глобальные зависимости (например, связь между ушами и хвостом собаки на изображении).










In [ ]:
!pip install timm torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from PIL import Image
from torchvision import transforms
import timm

In [ ]:
# Загрузка модели (предобученная Swin-Tiny)
model = timm.create_model('swin_tiny_patch4_window7_224', pretrained=True)
model.eval()  # Переводим в режим предсказания

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

SwinTransformer(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
    (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
  )
  (layers): Sequential(
    (0): SwinTransformerStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): SwinTransformerBlock(
          (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (attn): WindowAttention(
            (qkv): Linear(in_features=96, out_features=288, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=96, out_features=96, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (softmax): Softmax(dim=-1)
          )
          (drop_path1): Identity()
          (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU(approximate='none')
            (drop1): 

In [ ]:
# Трансформации для изображения
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Загрузка изображения (замените на свое)
img_path = 'dog.jpg'
img = Image.open(img_path).convert('RGB')
img_tensor = transform(img).unsqueeze(0)  # Добавляем размерность батча

In [ ]:
# Предсказание
with torch.no_grad():
    output = model(img_tensor)

In [ ]:
# Декодируем предсказание
probabilities = torch.nn.functional.softmax(output[0], dim=0)
top3_prob, top3_catid = torch.topk(probabilities, 3)

In [ ]:
if hasattr(model, 'get_classifier'):
    print("Используем метки из модели")
    for i in range(top3_prob.size(0)):
        print(f"Топ-{i+1}: Класс {top3_catid[i].item()} ({top3_prob[i].item():.2f})")
else:
  for i in range(top3_prob.size(0)):
        print(f"Топ-{i+1}: {classes[top3_catid[i]]} ({top3_prob[i].item():.2f})")

Используем метки из модели
Топ-1: Класс 159 (0.32)
Топ-2: Класс 211 (0.21)
Топ-3: Класс 180 (0.09)
